In [2]:
import polars as pl
import pandas as pd 
import os
import re
import glob
from urllib.request import urlopen, urlretrieve
from zipfile import ZipFile
from rarfile import RarFile
from bs4 import BeautifulSoup
import difflib
from br_denatran_frota.utils import guess_header, change_df_header, get_year_month_from_filename, verify_total, get_city_name_ibge, match_ibge
from constants import DICT_UFS, REGRAS

ImportError: cannot import name 'get_city_name_ibge' from 'utils' (/home/tamir/basedosdados/pipelines/br_denatran_frota/code/utils.py)

In [ ]:

#treat_uf_tipo('../files/2022/frota_por_uf_e_tipo_de_veículo_2-2022.xls')

In [ ]:
from download_frota import download_frota

In [ ]:
download_frota(2, 2022)

Download of frota_por_uf_e_tipo_de_veículo_2-2022.xls complete
Download of frota_por_município_e_tipo_2-2022.xls complete


In [ ]:
import basedosdados as bd 
municipios_query = """SELECT nome, id_municipio, sigla_uf FROM `basedosdados.br_bd_diretorios_brasil.municipio`
"""
bd_municipios = bd.read_sql(municipios_query, 'tamir-pipelines')
bd_municipios = pl.from_pandas(bd_municipios)

Downloading: 100%|██████████| 5570/5570 [00:00<00:00, 12793.96rows/s]


In [ ]:
file = '../files/2022/frota_por_município_e_tipo_2-2022.xls'
filename = os.path.split(file)[1]
df = pd.read_excel(file)
new_df  = change_df_header(df, guess_header(df))
new_df.rename(columns={ new_df.columns[0]: "sigla_uf" , new_df.columns[1]: 'nome_denatran'}, inplace= True) # Rename for ease of use.
new_df.sigla_uf = new_df.sigla_uf.str.strip() # Remove whitespace.
new_df = pl.from_pandas(new_df)

In [ ]:
uf = 'RN'

In [ ]:
# for uf in DICT_UFS:
denatran_uf = new_df.filter(pl.col('sigla_uf') == uf)
ibge_uf = bd_municipios.filter(pl.col('sigla_uf') == uf)
ibge_uf = ibge_uf.with_columns(pl.col('nome').str.to_lowercase())

In [ ]:
denatran_uf

sigla_uf,nome_denatran,TOTAL,AUTOMOVEL,BONDE,CAMINHAO,CAMINHAO TRATOR,CAMINHONETE,CAMIONETA,CHASSI PLATAF,CICLOMOTOR,MICRO-ONIBUS,MOTOCICLETA,MOTONETA,ONIBUS,QUADRICICLO,REBOQUE,SEMI-REBOQUE,SIDE-CAR,OUTROS,TRATOR ESTEI,TRATOR RODAS,TRICICLO,UTILITARIO
str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""RN""","""ACARI""",4035,1394,0,122,7,279,91,0,73,13,1684,286,16,0,31,8,0,15,0,0,4,12
"""RN""","""AFONSO BEZERRA…",1867,730,0,37,1,153,30,0,2,10,807,65,17,0,6,1,0,4,0,0,0,4
"""RN""","""AGUA NOVA""",1057,260,0,16,0,42,9,0,2,8,613,94,2,0,3,1,0,2,0,0,0,5
"""RN""","""ALEXANDRIA""",6122,2224,0,146,19,550,62,0,2,50,2686,216,18,0,48,34,0,25,0,0,1,41
"""RN""","""ALMINO AFONSO""",1621,465,0,24,1,95,15,0,4,9,843,135,3,0,10,4,0,6,0,0,0,7
"""RN""","""ALTO DO RODRIG…",5411,1898,0,266,7,408,73,1,15,22,2196,367,34,0,45,13,0,28,0,3,1,34
"""RN""","""ANGICOS""",4189,1400,0,139,5,424,72,0,18,21,1847,153,20,0,40,9,0,14,0,0,1,26
"""RN""","""ANTONIO MARTIN…",2016,550,0,46,0,122,20,0,2,13,1115,121,5,0,6,0,0,5,0,0,0,11
"""RN""","""APODI""",17771,4506,0,512,59,1051,123,0,361,54,8072,2683,63,0,71,46,0,84,0,0,1,85


In [ ]:

x = denatran_uf['nome_denatran'].apply(get_city_name_ibge)
denatran_uf = denatran_uf.with_columns(x.alias('suggested_nome_ibge'))

In [ ]:
match_ibge(denatran_uf, ibge_uf)

ValueError: Os seguintes municípios falharam: 
ASSU (RN)


In [ ]:
nomes = ibge_uf['nome'].to_list()

In [ ]:
joined_df = denatran_uf.join(ibge_uf, left_on=['suggested_nome_ibge', 'sigla_uf'], right_on=['nome', 'sigla_uf'], how='left')

mismatched = joined_df.filter(pl.col('id_municipio').is_null())

In [ ]:
import pandas as pd
zdf = denatran_uf.to_pandas()

REGRAS = {('RN', 'ASSU'): 'açu'}

# Define a function to apply the ruleset
def get_name_c(row):
    key = (row['sigla_uf'], row['nome_denatran'])
    if key in REGRAS:
        return REGRAS[key]
    else:
        return row['suggested_nome_ibge']

# Apply the function to the DataFrame
zdf['suggested_nome_ibge'] = zdf.apply(get_name_c, axis=1)

# Print the modified DataFrame
print(zdf)


    sigla_uf   nome_denatran  TOTAL  AUTOMOVEL  BONDE  CAMINHAO  \
0         RN           ACARI   4035       1394      0       122   
1         RN  AFONSO BEZERRA   1867        730      0        37   
2         RN       AGUA NOVA   1057        260      0        16   
3         RN      ALEXANDRIA   6122       2224      0       146   
4         RN   ALMINO AFONSO   1621        465      0        24   
..       ...             ...    ...        ...    ...       ...   
162       RN          VARZEA   1230        470      0        21   
163       RN       VENHA-VER    850        270      0        17   
164       RN       VERA CRUZ   3989       1238      0       211   
165       RN          VICOSA    515        162      0         8   
166       RN       VILA FLOR    847        298      0        10   

     CAMINHAO TRATOR  CAMINHONETE  CAMIONETA  CHASSI PLATAF  ...  QUADRICICLO  \
0                  7          279         91              0  ...            0   
1                  1          153

In [ ]:
zdf.loc[zdf['nome_denatran'] == 'ASSU']

,sigla_uf,nome_denatran,TOTAL,AUTOMOVEL,BONDE,CAMINHAO,CAMINHAO TRATOR,CAMINHONETE,CAMIONETA,CHASSI PLATAF,...,QUADRICICLO,REBOQUE,SEMI-REBOQUE,SIDE-CAR,OUTROS,TRATOR ESTEI,TRATOR RODAS,TRICICLO,UTILITARIO,suggested_nome_ibge
11,RN,ASSU,26667,8400,0,960,57,1668,322,0,...,0,219,97,0,97,0,0,12,186,açu


In [ ]:
import polars as pl

# Create example polars DataFrame
zdf = pl.DataFrame({
    'sigla_uf': ['RN', 'MG', 'RS'],
    'nome_denatran': ['ASSU', 'BELO HORIZONTE', 'PORTO ALEGRE'],
    'suggested_nome_ibge': ['Unknown', 'Unknown', 'Unknown'],
    'id': [1, 2, 3]
})

# Define your ruleset as a dictionary

# Define a function to apply the ruleset


# Apply the function to the polars DataFrame
denatran_uf = denatran_uf.with_columns(denatran_uf.apply(fix_suggested_nome_ibge)['apply'].alias('suggested_nome_ibge'))



In [ ]:
denatran_uf.apply(get_suggested_nome_ibge)['apply']


apply
str
"""acari"""
"""afonso bezerra…"
"""água nova"""
"""alexandria"""
"""almino afonso"""
"""alto do rodrig…"
"""angicos"""
"""antônio martin…"
"""apodi"""
